In [25]:
# Importamos las librerias necesarias.
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [26]:
# Creamos un DataFrame de nuestros datos.
car_data = pd.read_csv('../data/cars.csv', sep=';')

# Mostramos la forma y los primeros registros de nuestro DataFrame.
print(car_data.shape)
car_data.head()

(58049, 25)


,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,REV_Garantia,Averia_grave,QUEJA_CAC,EDAD_COCHE,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad Cliente,Tiempo
0,CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,...,NO DATA,Averia muy grave,SI,2.08,2892,0,0,2,18,0
1,CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,...,SI,No,NO,2.45,1376,7187,0,2,53,0
2,CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,...,NO DATA,No,NO,2.51,1376,0,1,4,21,3
3,CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,...,SI,Averia muy grave,SI,2.43,2015,7256,1,4,48,5
4,CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,...,NO DATA,No,NO,2.30,1818,0,1,3,21,3


In [27]:
# Revisamos si hay valores nulos en nuestro DataFrame.
valores_nulos = car_data.isnull().sum()

# Mostramos los valores nulos.
valores_nulos

CODE                   0
PRODUCTO               0
TIPO_CARROCERIA        0
COMBUSTIBLE            0
Potencia_              0
TRANS                  0
FORMA_PAGO             0
ESTADO_CIVIL         890
GENERO               860
OcupaciOn              0
PROVINCIA              0
Campanna1              0
Campanna2              0
Campanna3              0
Zona _Renta        13178
REV_Garantia           0
Averia_grave           1
QUEJA_CAC              0
EDAD_COCHE             0
COSTE_VENTA            0
km_anno                0
Mas_1_coche            0
Revisiones             0
Edad Cliente           0
Tiempo                 0
dtype: int64

In [28]:
# Revisamos si hay valores duplicados en nuestro DataFrame.
duplicados = car_data.duplicated().sum()

# Mostramos los valores duplicados.
duplicados

0

In [29]:
# Revisamos los tipos de datos de nuestro DataFrame.
tipos_de_datos = car_data.dtypes

# Mostramos los tipos de datos.
tipos_de_datos

CODE                object
PRODUCTO            object
TIPO_CARROCERIA     object
COMBUSTIBLE         object
Potencia_           object
TRANS               object
FORMA_PAGO          object
ESTADO_CIVIL        object
GENERO              object
OcupaciOn           object
PROVINCIA           object
Campanna1           object
Campanna2           object
Campanna3           object
Zona _Renta         object
REV_Garantia        object
Averia_grave        object
QUEJA_CAC           object
EDAD_COCHE         float64
COSTE_VENTA          int64
km_anno              int64
Mas_1_coche          int64
Revisiones           int64
Edad Cliente         int64
Tiempo               int64
dtype: object

Pasamos a numéricos las columnas que son de tipo object usando onehot encoder.

In [30]:
# Almacenar las columnas categóricas en una variable
cols_categoricas = car_data.select_dtypes(include=['object']).columns

# Mostrar los valores únicos de las columnas categóricas
for col in cols_categoricas:
    print(f"Valores únicos en la columna '{col}':")
    print(car_data[col].unique())
    print()  # Salto de línea para mejor visualización

Valores únicos en la columna 'CODE':
['CLI1' 'CLI2' 'CLI3' ... 'CLI57952' 'CLI58038' 'CLI58048']

Valores únicos en la columna 'PRODUCTO':
['A' 'B' 'C' 'H' 'J' 'D' 'I' 'E' 'F' 'K' 'G']

Valores únicos en la columna 'TIPO_CARROCERIA':
['TIPO1' 'TIPO6' 'TIPO4' 'TIPO7' 'TIPO8' 'TIPO2' 'TIPO3' 'TIPO5']

Valores únicos en la columna 'COMBUSTIBLE':
['FUEL 1' 'FUEL 2']

Valores únicos en la columna 'Potencia_':
['Baja' 'Media' 'Alta']

Valores únicos en la columna 'TRANS':
['M' 'A']

Valores únicos en la columna 'FORMA_PAGO':
['Contado' 'Otros' 'Financiera Marca' 'Financiera Banco']

Valores únicos en la columna 'ESTADO_CIVIL':
['CASADO' 'SOLTERO' 'OTROS' 'EN PAREJA' nan]

Valores únicos en la columna 'GENERO':
['M' 'F' nan]

Valores únicos en la columna 'OcupaciOn':
['Empresa' 'Funcionario' 'Autonomo']

Valores únicos en la columna 'PROVINCIA':
['Asturias' 'Toledo' 'Lerida' 'Madrid' 'Santa Cruz de Tenerife'
 'Pontevedra' 'Lacoruna' 'Barcelona' 'Cordoba' 'Guipuzcua' 'Valladolid'
 'Castellon' 

In [31]:
#Eliminamos las columnas

car_data = car_data.drop('CODE', axis=1)

car_data


,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,REV_Garantia,Averia_grave,QUEJA_CAC,EDAD_COCHE,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad Cliente,Tiempo
0,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,Asturias,...,NO DATA,Averia muy grave,SI,2.08,2892,0,0,2,18,0
1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,Toledo,...,SI,No,NO,2.45,1376,7187,0,2,53,0
2,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,Lerida,...,NO DATA,No,NO,2.51,1376,0,1,4,21,3
3,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,Madrid,...,SI,Averia muy grave,SI,2.43,2015,7256,1,4,48,5
4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,Santa Cruz de Tenerife,...,NO DATA,No,NO,2.30,1818,0,1,3,21,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58044,F,TIPO3,FUEL 1,Alta,M,Contado,SOLTERO,F,Empresa,Madrid,...,SI,Averia muy grave,SI,8.35,6832,12528,0,5,38,0
58045,K,TIPO7,FUEL 1,Alta,A,Contado,NaN,NaN,Empresa,Alicante,...,NO DATA,No,SI,7.68,0,22072,0,3,54,0
58046,K,TIPO7,FUEL 1,Alta,A,Contado,CASADO,M,Empresa,Madrid,...,NO DATA,No,NO,6.57,0,8352,0,8,63,0
58047,G,TIPO5,FUEL 2,Media,M,Contado,CASADO,M,Empresa,Zaragoza,...,NO DATA,No,NO,9.26,2786,10441,0,13,62,0


In [32]:
cols_onehotencoder = car_data[['PRODUCTO', 'TIPO_CARROCERIA', 'COMBUSTIBLE', 'TRANS', 'FORMA_PAGO', 'ESTADO_CIVIL', 'GENERO', 'OcupaciOn', 'PROVINCIA', 'Campanna1', 'Campanna2', 'Campanna3', 'REV_Garantia', 'QUEJA_CAC', 'Averia_grave']]

cols_labelencoder = car_data[['Potencia_', 'Zona _Renta']]

In [33]:
# Crear el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Ajustar el OneHotEncoder y transformar las columnas categóricas
encoded_data = encoder.fit_transform(cols_onehotencoder)

# Convertir los datos codificados en un DataFrame con los nombres de las columnas
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(cols_onehotencoder.columns))

# Concatenar las columnas codificadas al DataFrame original
df_final = pd.concat([car_data, encoded_df], axis=1)

# Eliminar las columnas originales categóricas si ya no son necesarias
df_final = df_final.drop(cols_onehotencoder.columns, axis=1)

# Mostrar el DataFrame final con las columnas One-Hot Encoded
df_final


,Potencia_,Zona _Renta,EDAD_COCHE,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad Cliente,Tiempo,PRODUCTO_A,...,Campanna3_SI,REV_Garantia_NO DATA,REV_Garantia_SI,QUEJA_CAC_NO,QUEJA_CAC_SI,Averia_grave_Averia grave,Averia_grave_Averia leve,Averia_grave_Averia muy grave,Averia_grave_No,Averia_grave_nan
0,Baja,Medio-Bajo,2.08,2892,0,0,2,18,0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,Baja,Medio-Bajo,2.45,1376,7187,0,2,53,0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Baja,Medio,2.51,1376,0,1,4,21,3,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Baja,Medio,2.43,2015,7256,1,4,48,5,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,Baja,Alto,2.30,1818,0,1,3,21,3,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58044,Alta,NaN,8.35,6832,12528,0,5,38,0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
58045,Alta,Medio-Bajo,7.68,0,22072,0,3,54,0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
58046,Alta,Medio,6.57,0,8352,0,8,63,0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
58047,Media,Medio-Bajo,9.26,2786,10441,0,13,62,0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
# Asegurarse de que no hay espacios adicionales en los nombres de las columnas
car_data.columns = car_data.columns.str.strip()

# Crear un objeto LabelEncoder
label_encoder = LabelEncoder()

# Aplicar LabelEncoder a cada columna
for col in cols_labelencoder:
    car_data[col] = label_encoder.fit_transform(car_data[col].astype(str))  #Convertir a string para manejar NaNs

# Mostrar las columnas codificadas
car_data[cols_labelencoder].head()

ValueError: Boolean array expected for the condition, not object

Identificamos putliers de 2 maneras: IQR y Z-score.

Método IQR: Los valores fuera del rango [𝑄1 − 1.5 × 𝐼𝑄𝑅, 𝑄3 + 1.5 × 𝐼𝑄𝑅] se consideran outliers.
Método Z-score: Los valores con un z-score absoluto mayor a 3 se consideran outliers (a más de 3 desviaciones estándar de la media).

In [ ]:
# Calcular el IQR para identificar outliers
Q1 = car_data.quantile(0.25)  # Primer cuartil (25%)
Q3 = car_data.quantile(0.75)  # Tercer cuartil (75%)
IQR = Q3 - Q1           # Rango intercuartílico

# Definir los límites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Encontrar los outliers
outliers = (car_data < lower_bound) | (car_data > upper_bound)

# Mostrar las filas que contienen outliers
print(car_data[outliers.any(axis=1)])

           CODE PRODUCTO TIPO_CARROCERIA COMBUSTIBLE Potencia_ TRANS  \
302      CLI303        A           TIPO1      FUEL 1      Baja     M   
1296    CLI1297        A           TIPO1      FUEL 2     Media     M   
1675    CLI1676        B           TIPO1      FUEL 1      Baja     M   
1739    CLI1740        B           TIPO1      FUEL 1     Media     M   
1908    CLI1909        B           TIPO1      FUEL 1     Media     M   
...         ...      ...             ...         ...       ...   ...   
57913  CLI55887        I           TIPO7      FUEL 2     Media     A   
58033  CLI57672        E           TIPO2      FUEL 1     Media     M   
58034  CLI57679        E           TIPO2      FUEL 1     Media     M   
58044  CLI57905        F           TIPO3      FUEL 1      Alta     M   
58047  CLI58038        G           TIPO5      FUEL 2     Media     M   

             FORMA_PAGO ESTADO_CIVIL GENERO OcupaciOn  ... REV_Garantia  \
302               Otros       CASADO      F   Empresa  ...  

C:\Users\Laura Rodríguez\AppData\Local\Temp\ipykernel_6452\2927218837.py:11: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers = (car_data < lower_bound) | (car_data > upper_bound)


In [ ]:
# Calcular los z-scores
z_scores = stats.zscore(car_data)

# Identificar los outliers (z-score > 3 o z-score < -3)
outliers_z = (abs(z_scores) > 3)

# Mostrar las filas que contienen outliers
print(car_data[outliers_z.any(axis=1)])

TypeError: can only concatenate str (not "float") to str